# Convolutional Neural Network

### Importing the libraries

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [5]:
tf.__version__

'2.3.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

#### We apply these transformations to avoid overfitting 
#### rescale = 1./255 -> This will apply feature scaling to all the pixels, as pixel values range between 0 and 255, so dividing all by 255, we can get all pixel values to be between 0 and 1.
#### shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True -> geometrical transformation ####
                                

In [6]:
# create object of ImageDataGenerator class
train_datagen = ImageDataGenerator(
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [8]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

#### Here we add a convolutional layer to our CNN####

In [11]:
# activation='relu'for rectifier activation function
# If we had black and white images, input_shape=[64, 64, 1]
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

#### Here we add a pooling layer to the convolutional layer####

In [12]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer with max pooling applied

In [13]:
# Remove input_shape=[64, 64, 3] as, this is only applied for the first input layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [14]:
# Flatten to a one dimensional layer
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

#### Used the flattened layer as the input layer for a new fully connected layer

In [16]:
# Larger number of hidden neurons = 128, to get better accuracy
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [18]:
# Sigmoid activation function for binary classification output layer
# Just one neuron needed as we want binary classfication so units = 1
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [20]:
# Arguments - 
# 'optimizer' - We choose 'adam' optimizer as it can perform stochastic gradient descent. 
#  The stochastic gradient descent will optimize the weights to minimize the loss between the actual and the predicted values.
# 'loss' - When we are doing a binary classification when we are predicting a binary variable use 'binary_crossentropy' and 
#  if we had categorical variable to predict we would have used 'categorical_crossentropy'
# 'metrics' -
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [21]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Epoch 1/30
250/250 [==============================] - 51s 206ms/step - loss: 0.6933 - accuracy: 0.5013 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/30
250/250 [==============================] - 46s 185ms/step - loss: 0.6932 - accuracy: 0.4947 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/30
250/250 [==============================] - 47s 189ms/step - loss: 0.6933 - accuracy: 0.4985 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/30
250/250 [==============================] - 52s 206ms/step - loss: 0.6932 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/30
250/250 [==============================] - 48s 191ms/step - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/30
250/250 [==============================] - 46s 184ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/30
250/250 [==============================] - 45s 180ms/step - loss: 0.6932 - accuracy: 0.4965 - val_loss: 0.6931 - val_ac

## Part 4 - Making a single prediction

In [22]:
import numpy as np
from keras.preprocessing import image
# image has to have size same as that one used during training test so target_size = (64, 64
# For a new image just add it in folder, and change name here to the image to predict results
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# predict method takes 2D arrays so convert image to numpy array
test_image = image.img_to_array(test_image)
# image has to be in a batch as we used before, even though its just one image
test_image = np.expand_dims(test_image, axis = 0)
# Now test_image has the right format to be accepted by predict()
result = cnn.predict(test_image)
# Run this to get indeces for dog = 1, cat = 0
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [23]:
print(prediction)

cat
